## Imports

In [124]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [125]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Sep. 1st Sep. 8th


In [126]:
filename = glob.glob('redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [127]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [128]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [130]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [131]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,September-6-2024,Condo/Co-op,7501 E Treasure Dr Unit 8K,North Bay Village,FL,33141.0,366000.0,2.0,2.0,TREASURES ON THE BAY II C,1130.0,NaN,1965.0,NaN,324.0,928.0,Sold,NaN,NaN,https://www.redfin.com/FL/North-Bay-Village/75...,MARMLS,A11573534,N,Y,25.845038,-80.145239


In [132]:
new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
          'PROPERTY TYPE':'Condo/Co-op',
          'ADDRESS':'PLACEHOLDER',
          'CITY':'PLACEHOLDER',
          'STATE OR PROVINCE':'FL',
          'ZIP OR POSTAL CODE':'PLACEHOLDER',
          'PRICE':99999999999,
          'BEDS':9999999999,
          'BATHS':999999999,
          'LOCATION':'PLACEHOLDER',
          'SQUARE FEET':9999999999,
          'LOT SIZE':None,
          'YEAR BUILT':'PLACEHOLDER',
          'DAYS ON MARKET':None,
          '$/SQUARE FEET':9999999999,
          'HOA/MONTH':999999999,
          'STATUS':'Sold',
          'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
          'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
          'URL':'Manual Entry',
          'SOURCE':'Manual Entry',
          'MLS#':'Manual Entry',
          'FAVORITE':'N',
          'INTERESTED':'Y',
          'LATITUDE':99.999999999,
          'LONGITUDE':99.999999999}

In [61]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [7]:
# # Filter dates between. If only using data looking back one week, this shouldn't change the df
# df = df[(df['SOLD DATE'] >= '2024-08-26') & (df['SOLD DATE'] <= '2024-09-02')]

In [62]:
df.sort_values(ascending=False, by='SOLD DATE')

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
2,PAST SALE,2024-09-09,Condo/Co-op,234 NE 3rd St #2202,Miami,FL,33132.0,480000.0,2.0,2.0,THE LOFT DOWNTOWN CONDO,1078.0,NaN,2005.0,NaN,445.0,1100.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/234-NE-3rd-St-...,MARMLS,A11655571,N,Y,25.776895,-80.189708
58,PAST SALE,2024-09-09,Condo/Co-op,9 Island Ave #1402,Miami Beach,FL,33139.0,950000.0,2.0,2.0,NINE ISLAND AVENUE CONDO,1154.0,NaN,1981.0,NaN,823.0,1602.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/9-Island...,MARMLS,A11572613,N,Y,25.790382,-80.148759
1,PAST SALE,2024-09-06,Condo/Co-op,7501 E Treasure Dr Unit 8K,North Bay Village,FL,33141.0,366000.0,2.0,2.0,TREASURES ON THE BAY II C,1130.0,NaN,1965.0,NaN,324.0,928.0,Sold,NaN,NaN,https://www.redfin.com/FL/North-Bay-Village/75...,MARMLS,A11573534,N,Y,25.845038,-80.145239
60,PAST SALE,2024-09-06,Condo/Co-op,5800 SW 127th Ave #2214,Miami,FL,33183.0,280000.0,2.0,2.0,MILLER GARDENS CONDO,1170.0,NaN,1981.0,NaN,239.0,519.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/5800-SW-127th-...,MARMLS,A11588632,N,Y,25.713249,-80.399863
25,PAST SALE,2024-09-06,Condo/Co-op,41 SE 5th St #808,Miami,FL,33131.0,630000.0,2.0,2.0,BRICKELL ON THE RIVER S T,1402.0,NaN,2006.0,NaN,449.0,1230.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/41-SE-5th-St-3...,MARMLS,A11604448,N,Y,25.769062,-80.191815
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,PAST SALE,2024-09-03,Condo/Co-op,20 Island Ave #618,Miami Beach,FL,33139.0,1190000.0,2.0,2.0,BELLE PLAZA CONDO,1534.0,NaN,1962.0,NaN,776.0,1757.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/20-Islan...,MARMLS,A11582338,N,Y,25.790260,-80.146413
63,PAST SALE,2024-09-03,Condo/Co-op,1265 W 24th St #102,Hialeah,FL,33010.0,195000.0,1.0,1.0,BELLA LUNA CONDO,626.0,NaN,1971.0,NaN,312.0,250.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hialeah/1265-W-24th-...,MARMLS,A11548760,N,Y,25.843710,-80.307603
64,PAST SALE,2024-09-03,Condo/Co-op,9999 Collins Ave Unit 19D,Bal Harbour,FL,33154.0,5250000.0,3.0,4.5,BAL HARBOUR TOWER CONDO,5210.0,NaN,1990.0,NaN,1008.0,7538.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/9999-Col...,MARMLS,A11552341,N,Y,25.891954,-80.122707
91,PAST SALE,2024-09-03,Condo/Co-op,155 Ocean Lane Dr #402,Key Biscayne,FL,33149.0,660000.0,1.0,1.5,COMMODORE CLUB WEST CONDO,895.0,NaN,1973.0,NaN,737.0,1154.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/155-Oce...,MARMLS,A11272464,N,Y,25.700100,-80.158645


In [65]:
df = df.sort_values(by='PRICE',ascending=False)

In [69]:
# df = df.drop(index=64)

In [70]:
df_top_ten = df.head(10)

## Collect Agent Information

In [71]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [72]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [73]:
response_df = pd.DataFrame(response_list)

In [74]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [75]:
df_top_ten = merged_df

In [186]:
# columns_to_drop = ['SALE TYPE','PROPERTY TYPE','STATE OR PROVINCE','ZIP OR POSTAL CODE','LOCATION',
#                    'LOT SIZE','DAYS ON MARKET',
#                   'HOA/MONTH','STATUS','NEXT OPEN HOUSE START TIME','NEXT OPEN HOUSE END TIME','URL',
#                   'FAVORITE','INTERESTED','url','list_broker_email_1',
#                    'list_broker_email_2','buy_broker_email_1','buy_broker_email_2','latitude','longitude',
#                   'latest_price','num_full_baths','num_partial_baths','price_per_sq_ft','sq_ft','year_built',
#                   'assembled_address','data_source_name','mls_id']

# chart_df = merged_df.drop(columns=columns_to_drop)

# chart_df = chart_df.rename(columns={'SOLD DATE':'Close Date','ADDRESS':'Address','BEDS':'Beds','BATHS':'Baths',
#                          'PRICE':'Price','SQUARE FEET':'Square Feet','YEAR BUILT':'Year Built',
#                         '$/SQUARE FEET':'PSF','list_agent_name_1':'List Agent 1',
#                          'list_broker_name_1':'List Broker 1','SOURCE':'Source',
#                         'list_agent_name_2':'List Agent 2', 'list_broker_name_2':'List Broker 2',
#                         'buy_agent_name_1':'Buy Agent 1','buy_broker_name_1':'Buy Broker 1',
#                         'buy_agent_name_2':'Buy Agent 2','buy_broker_name_2':'Buy Broker 2'})

# merged_df['ADDRESS'] = merged_df['ADDRESS'] + ' ' + merged_df['CITY']

## Current Week's Values

In [76]:
# Number of condos sold
current_week_condo_sales = len(df)
print(len(df))

90


In [77]:
# Average sale price
average_sale_price = round(df['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$592,142


In [78]:
# Average PSF
average_price_psf = round(df['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$475


In [79]:
# Total sales dollar volume
current_week_condo_sales_total = df['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$53,292,780


In [14]:
# # Define the date range
# start_date = '2024-05-20'
# end_date = '2024-05-26'

# # Filter rows where 'SOLD DATE' is within the specified range
# second_week_df = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

## Previous Week's Values

In [24]:
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

Input Previous Week Condo Sales Total (number units sold):
154
Input Previous Week Condo Average Sales Price:
693754
Input Previous Week Condo Average PSF:
480
Input Previous Week Condo Sales total (ex: 198_000_000)
107_000_000


## Clean Data

In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [80]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [81]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [82]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [83]:
pd.set_option('display.max_columns',None)

In [84]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-09-04,Condo/Co-op,18555 Collins Ave #3503,Sunny Isles Beach,FL,33160.0,4750000.0,3.0,4.5,Sunny Isles Beach,3785.0,NaN,2017.0,NaN,1255.0,5717.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A10277665,N,Y,25.948195,-80.12041,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Ronit Perez,Suma Luxury Realty,None,None,Jennifer Thomson,Coldwell Banker Realty,None,None,1,orange


In [85]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [86]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-09-04,Condo/Co-op,18555 Collins Ave #3503,Sunny Isles Beach,FL,33160.0,4750000.0,3.0,4.5,Sunny Isles Beach,3785.0,NaN,2017.0,NaN,1255.0,5717.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A10277665,N,Y,25.948195,-80.120410,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Ronit Perez,Suma Luxury Realty,None,None,Jennifer Thomson,Coldwell Banker Realty,None,None,1,orange
1,PAST SALE,2024-09-03,Condo/Co-op,4701 N Meridian Ave #303,Miami Beach,FL,33140.0,4290000.0,2.0,2.5,4701 NORTH MERIDIAN CONDO,2592.0,NaN,2019.0,NaN,1655.0,5764.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/4701-N-M...,MARMLS,A11471228,N,Y,25.822813,-80.130733,https://www.redfin.com/FL/Miami-Beach/4701-N-M...,Ronit Perez,Suma Luxury Realty,None,None,Jelena Khurana,The Corcoran Group,None,None,2,blue
2,PAST SALE,2024-09-06,Condo/Co-op,300 S Pointe Dr #1904,Miami Beach,FL,33139.0,2650000.0,2.0,2.5,PORTOFINO TOWER CONDO,2040.0,NaN,1996.0,NaN,1299.0,3482.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/300-S-Po...,MARMLS,A11573165,N,Y,25.767706,-80.134877,https://www.redfin.com/FL/Miami-Beach/300-S-Po...,JOHN Lennon,South Pointe Drive Realty Inc,None,None,Eyal Golan,Douglas Elliman,Debbie Golan,Douglas Elliman,3,blue
3,PAST SALE,2024-09-03,Condo/Co-op,601 NE 36th St #3201,Miami,FL,33137.0,1600000.0,3.0,3.5,BLUE CONDO,2413.0,NaN,2006.0,NaN,663.0,5252.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/601-NE-36th-St...,MARMLS,A11574391,N,Y,25.811114,-80.186240,https://www.redfin.com/FL/Miami/601-NE-36th-St...,Michelle Halfon,Sator Realty LLC,None,None,Marko Gojanovic,One Sotheby's International Re,None,None,4,blue
4,PAST SALE,2024-09-02,Condo/Co-op,900 Biscayne Blvd #5001,Miami,FL,33132.0,1295000.0,2.0,3.5,900 BISCAYNE BAY CONDO,1694.0,NaN,2008.0,NaN,764.0,2250.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/900-Biscayne-B...,MARMLS,A11634184,N,Y,25.783337,-80.190021,https://www.redfin.com/FL/Miami/900-Biscayne-B...,Alban Mecaj,"Nest Seekers Florida, LLC",None,None,Allen Tuller,OWN Realty,None,None,5,blue
5,PAST SALE,2024-09-03,Condo/Co-op,5875 Collins Ave #703,Miami Beach,FL,33140.0,1285000.0,2.0,2.5,MEI CONDO,1094.0,NaN,2008.0,NaN,1175.0,2496.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/5875-Col...,MARMLS,A11470672,N,Y,25.840891,-80.120220,https://www.redfin.com/FL/Miami-Beach/5875-Col...,Juliet Restrepo,Keller Williams Capital Realty,None,None,Benjamin Moss,"Compass Florida, LLC.",Mariacamila Castillo,"Compass Florida, LLC.",6,blue
6,PAST SALE,2024-09-03,Condo/Co-op,20 Island Ave #618,Miami Beach,FL,33139.0,1190000.0,2.0,2.0,BELLE PLAZA CONDO,1534.0,NaN,1962.0,NaN,776.0,1757.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/20-Islan...,MARMLS,A11582338,N,Y,25.790260,-80.146413,https://www.redfin.com/FL/Miami-Beach/20-Islan...,Scott Diffenderfer,"Compass Florida, LLC.",None,None,Felipe Cunha,London Foster Realty,None,None,7,blue
7,PAST SALE,2024-09-06,Condo/Co-op,610 Valencia Ave #301,Coral Gables,FL,33134.0,1150000.0,2.0,2.5,THE NEW FRENCH VILLAGE CO,1970.0,NaN,1991.0,NaN,584.0,1374.0,Sold,NaN,NaN,https://www.redfin.com/FL/Coral-Gables/610-Val...,MARMLS,A11637303,N,Y,25.747062,-80.267398,https://www.redfin.com/FL/Coral-Gables/610-Val...,Carole Smith,"Compass Florida, LLC.",Cristina Formosa,"Compass Florida, LLC.",Maggie Roche,Pawfriendly Homes Realty LLC,None,None,8,blue
8,PAST SALE,2024-09-03,Condo/Co-op,50 Ocean Lane Dr #201,Key Biscayne,FL,33149.0,1050000.0,3.0,2.5,PYRAMIDS KEY BISCAYNE,1730.0,NaN,1991.0,NaN,607.0,2134.0,Sold,NaN,NaN,https://www.redfin.com/

In [87]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} - {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [88]:
m.save('index.html')

## Data snagger

In [89]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [90]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-09-04,Condo/Co-op,18555 Collins Ave #3503,Sunny Isles Beach,FL,33160.0,4750000.0,3.0,4.5,Sunny Isles Beach,3785.0,NaN,2017.0,NaN,1255.0,5717.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A10277665,N,Y,25.948195,-80.12041,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Ronit Perez,Suma Luxury Realty,None,None,Jennifer Thomson,Coldwell Banker Realty,None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [91]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/condo_sales_week_ending_09092024


# CREATE TEMPLATE 

In [92]:
muni_set = set(df_top_ten['CITY'])

In [93]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [94]:
df_top_ten.reset_index(inplace=True,drop=True)

In [95]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [96]:
top_sale

'Sunny Isles Beach at 18555 Collins Ave #3503 in Sunny Isles Beach'

In [97]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [98]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [123]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-09-04,Condo/Co-op,18555 Collins Ave #3503,Sunny Isles Beach,FL,33160.0,4750000.0,3.0,4.5,Sunny Isles Beach,3785.0,NaN,2017.0,NaN,1255.0,5717.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A10277665,N,Y,25.948195,-80.120410,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Ronit Perez,Suma Luxury Realty,None,None,Jennifer Thomson,Coldwell Banker Realty,None,None,1,orange,Sunny Isles Beach at 18555 Collins Ave #3503 i...
1,PAST SALE,2024-09-03,Condo/Co-op,4701 N Meridian Ave #303,Miami Beach,FL,33140.0,4290000.0,2.0,2.5,4701 NORTH MERIDIAN CONDO,2592.0,NaN,2019.0,NaN,1655.0,5764.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/4701-N-M...,MARMLS,A11471228,N,Y,25.822813,-80.130733,https://www.redfin.com/FL/Miami-Beach/4701-N-M...,Ronit Perez,Suma Luxury Realty,None,None,Jelena Khurana,The Corcoran Group,None,None,2,blue,4701 NORTH MERIDIAN CONDO at 4701 N Meridian A...
2,PAST SALE,2024-09-06,Condo/Co-op,300 S Pointe Dr #1904,Miami Beach,FL,33139.0,2650000.0,2.0,2.5,PORTOFINO TOWER CONDO,2040.0,NaN,1996.0,NaN,1299.0,3482.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/300-S-Po...,MARMLS,A11573165,N,Y,25.767706,-80.134877,https://www.redfin.com/FL/Miami-Beach/300-S-Po...,JOHN Lennon,South Pointe Drive Realty Inc,None,None,Eyal Golan,Douglas Elliman,Debbie Golan,Douglas Elliman,3,blue,PORTOFINO TOWER CONDO at 300 S Pointe Dr #1904...
3,PAST SALE,2024-09-03,Condo/Co-op,601 NE 36th St #3201,Miami,FL,33137.0,1600000.0,3.0,3.5,BLUE CONDO,2413.0,NaN,2006.0,NaN,663.0,5252.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/601-NE-36th-St...,MARMLS,A11574391,N,Y,25.811114,-80.186240,https://www.redfin.com/FL/Miami/601-NE-36th-St...,Michelle Halfon,Sator Realty LLC,None,None,Marko Gojanovic,One Sotheby's International Re,None,None,4,blue,BLUE CONDO at 601 NE 36th St #3201 in Miami
4,PAST SALE,2024-09-02,Condo/Co-op,900 Biscayne Blvd #5001,Miami,FL,33132.0,1295000.0,2.0,3.5,900 BISCAYNE BAY CONDO,1694.0,NaN,2008.0,NaN,764.0,2250.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/900-Biscayne-B...,MARMLS,A11634184,N,Y,25.783337,-80.190021,https://www.redfin.com/FL/Miami/900-Biscayne-B...,Alban Mecaj,"Nest Seekers Florida, LLC",None,None,Allen Tuller,OWN Realty,None,None,5,blue,900 BISCAYNE BAY CONDO at 900 Biscayne Blvd #5...
5,PAST SALE,2024-09-03,Condo/Co-op,5875 Collins Ave #703,Miami Beach,FL,33140.0,1285000.0,2.0,2.5,MEI CONDO,1094.0,NaN,2008.0,NaN,1175.0,2496.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/5875-Col...,MARMLS,A11470672,N,Y,25.840891,-80.120220,https://www.redfin.com/FL/Miami-Beach/5875-Col...,Juliet Restrepo,Keller Williams Capital Realty,None,None,Benjamin Moss,"Compass Florida, LLC.",Mariacamila Castillo,"Compass Florida, LLC.",6,blue,MEI CONDO at 5875 Collins Ave #703 in Miami Beach
6,PAST SALE,2024-09-03,Condo/Co-op,20 Island Ave #618,Miami Beach,FL,33139.0,1190000.0,2.0,2.0,BELLE PLAZA CONDO,1534.0,NaN,1962.0,NaN,776.0,1757.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/20-Islan...,MARMLS,A11582338,N,Y,25.790260,-80.146413,https://www.redfin.com/FL/Miami-Beach/20-Islan...,Scott Diffenderfer,"Compass Florida, LLC.",None,None,Felipe Cunha,London Foster Realty,None,None,7,blue,BELLE PLAZA CONDO at 20 Island Ave #618 in Mia...
7,PAST SALE,2024-09-06,Condo/Co-op,610 Valencia Ave #301,Coral Gables,FL,33134.0,1150000.0,2.0,2.5,THE NEW FRENCH VILLAGE CO,1970.0,NaN,1991.0,NaN,584.0,1374.0,Sold,NaN,NaN,https://www.redfin.com/FL/Coral-Gables/610-Val...,MARMLS,A11637303,N,Y,25.747062,-80.267398,https://www.redfin.com/FL

In [122]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops Miami-Dade weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f}M to ${df_top_ten.at[0,'PRICE']:,.0f}M
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m Miami-Dade County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m Miami-Dade County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Adam Farence
\033[1mRESEARCH:\033[0m 
\033[1mSocial:\033[0m #MiamiDade condo sales XXXXX
\033[1mART:\033[0m

Interactive map

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, Miami-Dade County, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

Miami-Dade County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: Sunny Isles Beach closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $950,000M to $4,750,000M
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #MiamiDade condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade County, weekly condo sales, Sunny Isles Beach, Coral Gables, Miami Beach, Miami, Key Biscayne

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 90 condo sales totaling $53,292,780 million from Sep. 1st to Sep. 8th. The previous week, brokers closed 154 condo sales totaling $107,000,000.

Last week’s units sold for an average of $592,142, lower than the $693,754 average price from the previous week. The avera

In [100]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [101]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [102]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [121]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-09-04,Condo/Co-op,18555 Collins Ave #3503,Sunny Isles Beach,FL,33160.0,4750000.0,3.0,4.5,Sunny Isles Beach,3785.0,NaN,2017.0,NaN,1255.0,5717.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A10277665,N,Y,25.948195,-80.120410,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Ronit Perez,Suma Luxury Realty,None,None,Jennifer Thomson,Coldwell Banker Realty,None,None,1,orange,Sunny Isles Beach at 18555 Collins Ave #3503 i...
1,PAST SALE,2024-09-03,Condo/Co-op,4701 N Meridian Ave #303,Miami Beach,FL,33140.0,4290000.0,2.0,2.5,4701 NORTH MERIDIAN CONDO,2592.0,NaN,2019.0,NaN,1655.0,5764.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/4701-N-M...,MARMLS,A11471228,N,Y,25.822813,-80.130733,https://www.redfin.com/FL/Miami-Beach/4701-N-M...,Ronit Perez,Suma Luxury Realty,None,None,Jelena Khurana,The Corcoran Group,None,None,2,blue,4701 NORTH MERIDIAN CONDO at 4701 N Meridian A...
2,PAST SALE,2024-09-06,Condo/Co-op,300 S Pointe Dr #1904,Miami Beach,FL,33139.0,2650000.0,2.0,2.5,PORTOFINO TOWER CONDO,2040.0,NaN,1996.0,NaN,1299.0,3482.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/300-S-Po...,MARMLS,A11573165,N,Y,25.767706,-80.134877,https://www.redfin.com/FL/Miami-Beach/300-S-Po...,JOHN Lennon,South Pointe Drive Realty Inc,None,None,Eyal Golan,Douglas Elliman,Debbie Golan,Douglas Elliman,3,blue,PORTOFINO TOWER CONDO at 300 S Pointe Dr #1904...
3,PAST SALE,2024-09-03,Condo/Co-op,601 NE 36th St #3201,Miami,FL,33137.0,1600000.0,3.0,3.5,BLUE CONDO,2413.0,NaN,2006.0,NaN,663.0,5252.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/601-NE-36th-St...,MARMLS,A11574391,N,Y,25.811114,-80.186240,https://www.redfin.com/FL/Miami/601-NE-36th-St...,Michelle Halfon,Sator Realty LLC,None,None,Marko Gojanovic,One Sotheby's International Re,None,None,4,blue,BLUE CONDO at 601 NE 36th St #3201 in Miami
4,PAST SALE,2024-09-02,Condo/Co-op,900 Biscayne Blvd #5001,Miami,FL,33132.0,1295000.0,2.0,3.5,900 BISCAYNE BAY CONDO,1694.0,NaN,2008.0,NaN,764.0,2250.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/900-Biscayne-B...,MARMLS,A11634184,N,Y,25.783337,-80.190021,https://www.redfin.com/FL/Miami/900-Biscayne-B...,Alban Mecaj,"Nest Seekers Florida, LLC",None,None,Allen Tuller,OWN Realty,None,None,5,blue,900 BISCAYNE BAY CONDO at 900 Biscayne Blvd #5...
5,PAST SALE,2024-09-03,Condo/Co-op,5875 Collins Ave #703,Miami Beach,FL,33140.0,1285000.0,2.0,2.5,MEI CONDO,1094.0,NaN,2008.0,NaN,1175.0,2496.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/5875-Col...,MARMLS,A11470672,N,Y,25.840891,-80.120220,https://www.redfin.com/FL/Miami-Beach/5875-Col...,Juliet Restrepo,Keller Williams Capital Realty,None,None,Benjamin Moss,"Compass Florida, LLC.",Mariacamila Castillo,"Compass Florida, LLC.",6,blue,MEI CONDO at 5875 Collins Ave #703 in Miami Beach
6,PAST SALE,2024-09-03,Condo/Co-op,20 Island Ave #618,Miami Beach,FL,33139.0,1190000.0,2.0,2.0,BELLE PLAZA CONDO,1534.0,NaN,1962.0,NaN,776.0,1757.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/20-Islan...,MARMLS,A11582338,N,Y,25.790260,-80.146413,https://www.redfin.com/FL/Miami-Beach/20-Islan...,Scott Diffenderfer,"Compass Florida, LLC.",None,None,Felipe Cunha,London Foster Realty,None,None,7,blue,BELLE PLAZA CONDO at 20 Island Ave #618 in Mia...
7,PAST SALE,2024-09-06,Condo/Co-op,610 Valencia Ave #301,Coral Gables,FL,33134.0,1150000.0,2.0,2.5,THE NEW FRENCH VILLAGE CO,1970.0,NaN,1991.0,NaN,584.0,1374.0,Sold,NaN,NaN,https://www.redfin.com/FL/Coral-Gables/610-Val...,MARMLS,A11637303,N,Y,25.747062,-80.267398,https://www.redfin.com/FL

In [120]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}")

print(story_checklist)


HED: Sunny Isles Beach closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $950,000M to $4,750,000M
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #MiamiDade condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade County, weekly condo sales, Sunny Isles Beach, Coral Gables, Miami Beach, Miami, Key Biscayne

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 90 condo sales totaling $53,292,780 million from Sep. 1st to Sep. 8th. The previous week, brokers closed 154 condo sales totaling $107,000,000.

Last week’s units sold for an average of $592,142, lower than the $693,754 average price from the previous week. The avera

In [104]:
print(df_top_ten['URL'].iloc[0])

https://www.redfin.com/FL/Sunny-Isles-Beach/18555-Collins-Ave-33160/unit-3503/home/55274161


In [105]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Miami-Beach/4701-N-Meridian-Ave-33140/unit-303/home/79136261


In [106]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Miami-Beach/9-Island-Ave-33139/unit-1402/home/42790817


In [107]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Key-Biscayne/50-Ocean-Lane-Dr-33149/unit-201/home/42924390


In [108]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Miami-Beach/4701-N-Meridian-Ave-33140/unit-303/home/79136261


In [109]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Coral-Gables/610-Valencia-Ave-33134/unit-301/home/42812702


In [110]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-09-04,Condo/Co-op,18555 Collins Ave #3503,Sunny Isles Beach,FL,33160.0,4750000.0,3.0,4.5,Sunny Isles Beach,3785.0,NaN,2017.0,NaN,1255.0,5717.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A10277665,N,Y,25.948195,-80.120410,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Ronit Perez,Suma Luxury Realty,None,None,Jennifer Thomson,Coldwell Banker Realty,None,None,1,orange,Sunny Isles Beach at 18555 Collins Ave #3503 i...
1,PAST SALE,2024-09-03,Condo/Co-op,4701 N Meridian Ave #303,Miami Beach,FL,33140.0,4290000.0,2.0,2.5,4701 NORTH MERIDIAN CONDO,2592.0,NaN,2019.0,NaN,1655.0,5764.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/4701-N-M...,MARMLS,A11471228,N,Y,25.822813,-80.130733,https://www.redfin.com/FL/Miami-Beach/4701-N-M...,Ronit Perez,Suma Luxury Realty,None,None,Jelena Khurana,The Corcoran Group,None,None,2,blue,4701 NORTH MERIDIAN CONDO at 4701 N Meridian A...
2,PAST SALE,2024-09-06,Condo/Co-op,300 S Pointe Dr #1904,Miami Beach,FL,33139.0,2650000.0,2.0,2.5,PORTOFINO TOWER CONDO,2040.0,NaN,1996.0,NaN,1299.0,3482.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/300-S-Po...,MARMLS,A11573165,N,Y,25.767706,-80.134877,https://www.redfin.com/FL/Miami-Beach/300-S-Po...,JOHN Lennon,South Pointe Drive Realty Inc,None,None,Eyal Golan,Douglas Elliman,Debbie Golan,Douglas Elliman,3,blue,PORTOFINO TOWER CONDO at 300 S Pointe Dr #1904...
3,PAST SALE,2024-09-03,Condo/Co-op,601 NE 36th St #3201,Miami,FL,33137.0,1600000.0,3.0,3.5,BLUE CONDO,2413.0,NaN,2006.0,NaN,663.0,5252.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/601-NE-36th-St...,MARMLS,A11574391,N,Y,25.811114,-80.186240,https://www.redfin.com/FL/Miami/601-NE-36th-St...,Michelle Halfon,Sator Realty LLC,None,None,Marko Gojanovic,One Sotheby's International Re,None,None,4,blue,BLUE CONDO at 601 NE 36th St #3201 in Miami
4,PAST SALE,2024-09-02,Condo/Co-op,900 Biscayne Blvd #5001,Miami,FL,33132.0,1295000.0,2.0,3.5,900 BISCAYNE BAY CONDO,1694.0,NaN,2008.0,NaN,764.0,2250.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/900-Biscayne-B...,MARMLS,A11634184,N,Y,25.783337,-80.190021,https://www.redfin.com/FL/Miami/900-Biscayne-B...,Alban Mecaj,"Nest Seekers Florida, LLC",None,None,Allen Tuller,OWN Realty,None,None,5,blue,900 BISCAYNE BAY CONDO at 900 Biscayne Blvd #5...
5,PAST SALE,2024-09-03,Condo/Co-op,5875 Collins Ave #703,Miami Beach,FL,33140.0,1285000.0,2.0,2.5,MEI CONDO,1094.0,NaN,2008.0,NaN,1175.0,2496.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/5875-Col...,MARMLS,A11470672,N,Y,25.840891,-80.120220,https://www.redfin.com/FL/Miami-Beach/5875-Col...,Juliet Restrepo,Keller Williams Capital Realty,None,None,Benjamin Moss,"Compass Florida, LLC.",Mariacamila Castillo,"Compass Florida, LLC.",6,blue,MEI CONDO at 5875 Collins Ave #703 in Miami Beach
6,PAST SALE,2024-09-03,Condo/Co-op,20 Island Ave #618,Miami Beach,FL,33139.0,1190000.0,2.0,2.0,BELLE PLAZA CONDO,1534.0,NaN,1962.0,NaN,776.0,1757.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/20-Islan...,MARMLS,A11582338,N,Y,25.790260,-80.146413,https://www.redfin.com/FL/Miami-Beach/20-Islan...,Scott Diffenderfer,"Compass Florida, LLC.",None,None,Felipe Cunha,London Foster Realty,None,None,7,blue,BELLE PLAZA CONDO at 20 Island Ave #618 in Mia...
7,PAST SALE,2024-09-06,Condo/Co-op,610 Valencia Ave #301,Coral Gables,FL,33134.0,1150000.0,2.0,2.5,THE NEW FRENCH VILLAGE CO,1970.0,NaN,1991.0,NaN,584.0,1374.0,Sold,NaN,NaN,https://www.redfin.com/FL/Coral-Gables/610-Val...,MARMLS,A11637303,N,Y,25.747062,-80.267398,https://www.redfin.com/FL

## Time on Market Calculator

In [137]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 2, 27) ## List (Earlier) date
date2 = datetime(2024, 8, 27) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

182


## Clean CSV for Datawrapper Chart

In [111]:
chart_df = df_top_ten

In [112]:
chart_df = chart_df.fillna(" ")

In [113]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [114]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [115]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [116]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [117]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x))
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [118]:
csv_date_string = today.strftime("%m_%d_%Y")

In [119]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv")